In [1]:
%pip install -q supabase python-dotenv pandas numpy scipy matplotlib pillow

You should consider upgrading via the '/Users/rafaellopez/Desktop/DS3/tactical-viewer-app/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv
from scipy.spatial import KDTree
from supabase import create_client

load_dotenv()

url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_ANON_KEY")
if not url or not key:
    raise ValueError("Missing SUPABASE_URL or SUPABASE_ANON_KEY")

supabase = create_client(url, key)
print("Supabase connected")

Supabase connected


/Users/rafaellopez/Desktop/DS3/tactical-viewer-app/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
TABLE = "vw_voronoi_window_cr_sh"
SELECT_COLS = (
    "match_id,game_event_id,possession_event_id,possession_event_type,"
    "event_time_ms,frame_num,video_time_ms,side,jersey_num,x,y,visibility"
)

# Set this to a match_id with many events.
# Use None only if you really want all matches (very large).
MATCH_ID_FILTER = 3812

PAGE_SIZE = 1000
MAX_PAGES = 10000

def fetch_view_rows(client, table_name, select_cols, match_id=None, page_size=1000, max_pages=10000):
    out = []
    start = 0
    for i in range(max_pages):
        end = start + page_size - 1
        q = client.table(table_name).select(select_cols).range(start, end)
        if match_id is not None:
            q = q.eq("match_id", int(match_id))

        resp = q.execute()
        rows = resp.data or []
        n = len(rows)
        print(f"page {i+1}: +{n} rows (total {len(out)+n})")

        if n == 0:
            break

        out.extend(rows)
        start += n  # advance by actual returned rows

        if n < page_size:
            break

    return out

rows = fetch_view_rows(
    supabase, TABLE, SELECT_COLS,
    match_id=MATCH_ID_FILTER, page_size=PAGE_SIZE, max_pages=MAX_PAGES
)

if not rows:
    raise ValueError("No rows returned from view")

df_game = pd.DataFrame(rows)
print("rows:", len(df_game))

page 1: +1000 rows (total 1000)
page 2: +1000 rows (total 2000)
page 3: +1000 rows (total 3000)
page 4: +1000 rows (total 4000)
page 5: +1000 rows (total 5000)
page 6: +1000 rows (total 6000)
page 7: +1000 rows (total 7000)
page 8: +1000 rows (total 8000)
page 9: +1000 rows (total 9000)
page 10: +1000 rows (total 10000)
page 11: +1000 rows (total 11000)
page 12: +1000 rows (total 12000)
page 13: +1000 rows (total 13000)
page 14: +1000 rows (total 14000)
page 15: +1000 rows (total 15000)
page 16: +1000 rows (total 16000)
page 17: +1000 rows (total 17000)
page 18: +1000 rows (total 18000)
page 19: +1000 rows (total 19000)
page 20: +1000 rows (total 20000)
page 21: +1000 rows (total 21000)
page 22: +1000 rows (total 22000)
page 23: +1000 rows (total 23000)
page 24: +1000 rows (total 24000)
page 25: +1000 rows (total 25000)
page 26: +1000 rows (total 26000)
page 27: +1000 rows (total 27000)
page 28: +1000 rows (total 28000)
page 29: +1000 rows (total 29000)
page 30: +1000 rows (total 30000

In [4]:
for c in ["event_time_ms", "video_time_ms", "x", "y"]:
    df_game[c] = pd.to_numeric(df_game[c], errors="coerce")

df_game["possession_event_type"] = df_game["possession_event_type"].astype(str).str.upper()
df_game = df_game.dropna(subset=["video_time_ms", "x", "y"]).copy()

vis = df_game["visibility"].astype(str).str.upper()
df_game = df_game[vis.isin(["VISIBLE", "ESTIMATED", "TRACKED", "NONE", "NAN"])].copy()

def norm_side(v):
    s = str(v).strip().lower()
    if ("home" in s) or (s in {"h", "1"}):
        return "home"
    if ("away" in s) or (s in {"a", "2"}):
        return "away"
    return np.nan

df_game["side_norm"] = df_game["side"].map(norm_side)
df_game = df_game.dropna(subset=["side_norm"]).copy()

# robust event id (prevents collapse)
df_game["event_uid"] = (
    df_game["possession_event_type"].astype(str) + "_" +
    df_game["game_event_id"].astype(str) + "_" +
    df_game["event_time_ms"].round().astype("int64").astype(str)
)

# relative time around event (negative=before, positive=after)
df_game["rel_ms"] = df_game["video_time_ms"] - df_game["event_time_ms"]

print("unique events:", df_game["event_uid"].nunique())
display(
    df_game[["event_uid","possession_event_type"]]
    .drop_duplicates()
    .groupby("possession_event_type")["event_uid"]
    .nunique()
    .rename("n_events")
    .to_frame()
)

unique events: 76


,n_events
possession_event_type,
CR,49
SH,27


In [5]:
pitch_length, pitch_width = 105, 68
n_x_bins, n_y_bins = 100, 60

x_bins = np.linspace(-pitch_length/2, pitch_length/2, n_x_bins)
y_bins = np.linspace(-pitch_width/2, pitch_width/2, n_y_bins)
grid_x, grid_y = np.meshgrid(x_bins, y_bins)
grid_points = np.column_stack([grid_x.ravel(), grid_y.ravel()])
cell_area = (pitch_length / n_x_bins) * (pitch_width / n_y_bins)

def frame_voronoi_home_share(frame_players: pd.DataFrame):
    frame_players = frame_players.drop_duplicates(subset=["side_norm", "jersey_num"], keep="last")
    if frame_players.shape[0] < 2:
        return np.nan, np.nan, np.nan, int(frame_players.shape[0])

    positions = frame_players[["x", "y"]].to_numpy(dtype=float)
    labels = frame_players["side_norm"].to_numpy(dtype=str)

    if len(np.unique(labels)) < 2:
        return np.nan, np.nan, np.nan, int(frame_players.shape[0])

    tree = KDTree(positions)
    _, idx = tree.query(grid_points)
    assigned = labels[idx]

    home_cells = np.sum(assigned == "home")
    away_cells = np.sum(assigned == "away")

    home_area = home_cells * cell_area
    away_area = away_cells * cell_area
    total = home_area + away_area
    home_share = home_area / total if total > 0 else np.nan
    return home_share, home_area, away_area, int(frame_players.shape[0])

frame_rows = []
group_cols = ["event_uid", "possession_event_type", "video_time_ms", "rel_ms"]

for (event_uid, evt_type, vms, rel_ms), g in df_game.groupby(group_cols, sort=False):
    hs, ha, aa, nppl = frame_voronoi_home_share(g)
    frame_rows.append({
        "event_uid": event_uid,
        "event_type": evt_type,
        "video_time_ms": vms,
        "rel_ms": rel_ms,
        "home_share": hs,
        "home_area_m2": ha,
        "away_area_m2": aa,
        "n_players": nppl
    })

res = pd.DataFrame(frame_rows).dropna(subset=["home_share"]).copy()
res["rel_s"] = res["rel_ms"] / 1000.0

print("Valid Voronoi frames:", len(res))
print("Events with valid Voronoi:", res["event_uid"].nunique())

Valid Voronoi frames: 3774
Events with valid Voronoi: 76


In [7]:
from matplotlib.colors import ListedColormap

def build_frame_assignment(frame_players):
    frame_players = frame_players.drop_duplicates(subset=["side_norm", "jersey_num"], keep="last")
    frame_players = frame_players.dropna(subset=["x", "y", "side_norm"])
    if frame_players.shape[0] < 2:
        return None, None

    positions = frame_players[["x", "y"]].to_numpy(dtype=float)
    labels = frame_players["side_norm"].to_numpy(dtype=str)
    if len(np.unique(labels)) < 2:
        return None, None

    tree = KDTree(positions)
    _, idx = tree.query(grid_points)
    assigned_sides = labels[idx]

    assigned_int = (assigned_sides == "home").astype(int).reshape(grid_x.shape)
    return assigned_int, frame_players

In [8]:
from pathlib import Path
from matplotlib.animation import FuncAnimation, PillowWriter

OUTPUT_DIR = Path("../outputs/voronoi_event_clips")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = OUTPUT_DIR / "all_events_first_10.gif"
PAUSE_FRAMES = 6
DPI = 120
MAX_EVENTS = 10

event_ids = (
    res.sort_values("video_time_ms")["event_uid"]
    .dropna()
    .drop_duplicates()
    .head(MAX_EVENTS)
    .tolist()
)

timeline = []
event_frame_cache = {}

for event_id in event_ids:
    frames = (
        res[res["event_uid"] == event_id]
        .sort_values("video_time_ms")
        .reset_index(drop=True)
    )
    if len(frames) < 2:
        continue
    event_frame_cache[event_id] = frames
    for i in range(len(frames)):
        timeline.append((event_id, i))
    for _ in range(PAUSE_FRAMES):
        timeline.append((event_id, len(frames) - 1))

print("total frames in timeline:", len(timeline))

all_dt = res["video_time_ms"].sort_values().diff().dropna()
fps = int(np.clip(round(1000 / max(all_dt.median(), 1)), 5, 25)) if len(all_dt) else 12
print("fps:", fps)

fig, ax = plt.subplots(figsize=(11, 7))
im = ax.imshow(
    np.zeros_like(grid_x),
    origin="lower",
    extent=[-pitch_length/2, pitch_length/2, -pitch_width/2, pitch_width/2],
    aspect="auto",
    cmap=ListedColormap(["blue", "red"]),
    vmin=0, vmax=1, alpha=0.35
)
sc_home = ax.scatter([], [], c="red", s=70, edgecolor="black", label="home")
sc_away = ax.scatter([], [], c="blue", s=70, edgecolor="black", label="away")
txt = ax.text(
    0.01, 0.99, "", transform=ax.transAxes, va="top", ha="left",
    bbox=dict(facecolor="white", alpha=0.7, edgecolor="none")
)
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_title("Voronoi all events (first 10)")
ax.legend(loc="upper right")

def update(idx):
    event_id, frame_i = timeline[idx]
    frames = event_frame_cache[event_id]
    frame_ms = frames.loc[frame_i, "video_time_ms"]
    rel_s = frames.loc[frame_i, "rel_ms"] / 1000.0
    event_type = frames.loc[frame_i, "event_type"]

    fp = df_game[
        (df_game["event_uid"] == event_id) &
        (df_game["video_time_ms"] == frame_ms)
    ].copy()

    assigned_int, frame_players = build_frame_assignment(fp)
    if assigned_int is None:
        return im, sc_home, sc_away, txt

    im.set_data(assigned_int)

    home = frame_players[frame_players["side_norm"] == "home"][["x", "y"]].to_numpy()
    away = frame_players[frame_players["side_norm"] == "away"][["x", "y"]].to_numpy()
    sc_home.set_offsets(home if len(home) else np.empty((0, 2)))
    sc_away.set_offsets(away if len(away) else np.empty((0, 2)))

    txt.set_text(f"{event_type} | {event_id} | t={rel_s:.2f}s")
    return im, sc_home, sc_away, txt

anim = FuncAnimation(fig, update, frames=len(timeline), interval=int(1000 / fps), blit=False, repeat=False)
anim.save(OUT_PATH, writer=PillowWriter(fps=fps), dpi=DPI)
plt.close(fig)

print("Saved:", OUT_PATH.resolve())

total frames in timeline: 556
fps: 10
Saved: /Users/rafaellopez/Desktop/DS3/tactical-viewer-app/outputs/voronoi_event_clips/all_events_first_10.gif
